In [1]:
import csv
import json
import os
import requests

In [7]:
import requests
import json
import os
import csv

def fetch_manifest_data(manifest_url):
    """
    Fetches and returns JSON data from a given IIIF image manifest URL.
    """
    try:
        response = requests.get(manifest_url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching manifest from {manifest_url}: {e}")
        return None

def create_book_manifest(input_csv_path, column_name, output_path="book.json"):
    """
    Creates a valid IIIF 3.0 book manifest from a list of image manifests provided in a CSV file.

    Args:
        input_csv_path: Path to the CSV file containing manifest URLs.
        column_name: Name of the column containing manifest URLs.
        output_path: Path to save the generated book manifest (default: 'book.json').
    """
    # Initialize the book manifest
    book_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": "http://example.org/iiif/book.json",
        "type": "Manifest",
        "label": {"en": ["Generated IIIF Book Manifest"]},
        "items": []  # Canvases
    }

    # Read manifest URLs from the CSV file
    manifest_urls = []
    try:
        with open(input_csv_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if column_name in row:
                    manifest_urls.append(row[column_name])
                else:
                    print(f"Column '{column_name}' not found in the CSV file.")
                    return
    except FileNotFoundError:
        print(f"CSV file not found: {input_csv_path}")
        return

    # Process each manifest URL
    for manifest_url in manifest_urls:
        manifest_data = fetch_manifest_data(manifest_url)
        if not manifest_data:
            continue

        # Extract necessary properties from the manifest
        manifest_id = manifest_data.get("id", manifest_data.get("@id"))
        label = manifest_data.get("label", {"en": ["No label"]})
        width = manifest_data.get("width", 1000)  # Default width
        height = manifest_data.get("height", 1000)  # Default height

        # Ensure body is wrapped in an array as per IIIF 3.0 spec
        body = [
            {
                "id": f"{manifest_id}/image",
                "type": "Image",
                "format": manifest_data.get("format", "image/jpeg"),
                "width": width,
                "height": height
            }
        ]

        # Create the Canvas for this manifest
        canvas = {
            "id": f"{manifest_id}/canvas",
            "type": "Canvas",
            "label": label,
            "width": width,
            "height": height,
            "items": [
                {
                    "id": f"{manifest_id}/annotation-page",
                    "type": "AnnotationPage",
                    "items": [
                        {
                            "id": f"{manifest_id}/annotation",
                            "type": "Annotation",
                            "motivation": "painting",
                            "body": body,
                            "target": f"{manifest_id}/canvas"
                        }
                    ]
                }
            ]
        }

        # Append the canvas to the book manifest
        book_manifest["items"].append(canvas)

    # Save the book manifest to the output path
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(book_manifest, f, indent=2)
    print(f"Book manifest successfully created at: {output_path}")

def main():
    """
    Main function to run the script. Takes user input for CSV path, column name, and output file.
    """
    input_csv_path = input("Enter the path to the input CSV file: ").strip()
    column_name = input("Enter the column name containing manifest URLs: ").strip()
    output_path = input("Enter the desired output path for the book manifest (default: 'book.json'): ").strip()
    output_path = output_path if output_path else "book.json"

    create_book_manifest(input_csv_path, column_name, output_path)

if __name__ == "__main__":
    main()


Book manifest successfully created at: ./ouput/manifest.json
